# Reformat final dataset

Purpose of this notebook is to re-format the final ECG and EDA datasets from long into wide format

## Imports

In [ ]:
# fmt: off
import os
from pathlib import Path
import numpy as np
import pandas as pd
# fmt: on

## Parameters

In [ ]:
DATA_DIR = Path().cwd().parent / "data"
FINAL_DATA_DIR = DATA_DIR / "final"
BEHAVIORAL_DATA_DIR = DATA_DIR / "behavioral"

## Support Functions

In [ ]:
def prettify_pivot_colnames(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [
        f"{var}_{segment}" for var, segment in df.columns
    ]
    return df


def map_segment_story_code(row: pd.Series) -> str:
    if row["segment_name"] == "Story 1":
        return row["STORY1NAME"]
    elif row["segment_name"] == "Story 2":
        return row["STORY2NAME"]
    elif row["segment_name"] == "Story 3":
        return row["STORY3NAME"]
    elif row["segment_name"] == "Story 4":
        return row["STORY4NAME"]
    elif row["segment_name"] == "Story 5":
        return row["STORY5NAME"]
    else:
        return np.nan
    
def map_segment_story_score(row: pd.Series) -> str:
    if row["segment_name"] == "Story 1":
        return np.nan
    elif row["segment_name"] == "Story 2":
        return row["Story2_Score"]
    elif row["segment_name"] == "Story 3":
        return row["Story3_Score"]
    elif row["segment_name"] == "Story 4":
        return row["Story4_Score"]
    elif row["segment_name"] == "Story 5":
        return row["Story5_Score"]
    else:
        return np.nan

# Restructure ECG data

### Load required datasets

In [ ]:
extra_df = pd.read_excel(BEHAVIORAL_DATA_DIR / "filtered_data_v3 (with ASAs).xlsx")
print(f"{extra_df.shape = }")
extra_df.head(3)


In [ ]:
ecg_long_df = pd.read_excel(FINAL_DATA_DIR / "ecg_metrics" / "group_level_blc_ecg_metrics.xlsx").drop(columns = ["Unnamed: 0"])
print(f"{ecg_long_df.shape = }")
ecg_long_df.head(2)


### Re-format column names

In [ ]:
story_segment_df = ecg_long_df[["segment_name", "subject_id"]].merge(
    extra_df[["ParticipantID", "STORY1NAME", "STORY2NAME", "STORY3NAME", "STORY4NAME", "STORY5NAME", "Story2_Score", "Story3_Score", "Story4_Score", "Story5_Score"]],
    left_on = ["subject_id"],
    right_on = ["ParticipantID"],
    how = "right"
)
# story_segment_df
story_segment_df.loc[:, "story_order"] = story_segment_df['segment_name']
story_segment_df.loc[:, "story_score"] = story_segment_df.apply(map_segment_story_score, axis=1)
story_segment_df.loc[:, "segment_name"] = story_segment_df.apply(map_segment_story_code, axis=1)
story_segment_df.head(3)

In [ ]:
cleaned_ecg_long_df = (
    ecg_long_df
    # 
    .merge(story_segment_df[["segment_name", "subject_id", "story_order", "story_score"]], 
           left_on=["segment_name", "subject_id"], 
           right_on=["story_order", "subject_id"],
           how="inner")
    .drop(columns=["segment_name_x"])
    .rename(columns = {"segment_name_y": "story_name"})
)
cleaned_ecg_long_df.head()

### Pivot

In [ ]:
pivot_df = (
    cleaned_ecg_long_df
    .pivot(index = "subject_id", columns=["story_name"], values=[
        "HRV_SDNN_baseline", "HRV_SDNN_corrected", 
        "RSA_PorgesBohrer_baseline", "RSA_PorgesBohrer_corrected",
        "heart_rate_bpm", "heart_rate_bpm_corrected",
        "story_score"])
    )
pivot_df

### Combine AC and BH into ACBH

In [ ]:
story_pair = ('AC', 'BH')
new_story = 'ACBH'

# Select only top-level columns (the metrics)
metrics = pivot_df.columns.levels[0]

for metric in metrics:
    if story_pair[0] in pivot_df[metric] or story_pair[1] in pivot_df[metric]:
        pivot_df[(metric, new_story)] = pivot_df[(metric, story_pair[1])].combine_first(
            pivot_df[(metric, story_pair[0])]
        )
        pivot_df = pivot_df.drop(columns=[(metric, story_pair[0]), (metric, story_pair[1])])
pivot_df.head(5)

### Prettify colnames

In [ ]:
ecg_wide_df = prettify_pivot_colnames(pivot_df)
ecg_wide_df

# Restructure EDA data

In [ ]:
extra_df = pd.read_excel(BEHAVIORAL_DATA_DIR / "filtered_data_v3 (with ASAs).xlsx")
print(f"{extra_df.shape = }")
extra_df.head(3)

In [ ]:
eda_long_df = pd.read_excel(FINAL_DATA_DIR / "eda_features" / "group_level_blc_eda_features.xlsx").drop(columns = ["Unnamed: 0"])
print(f"{eda_long_df.shape = }")
eda_long_df.head(2)


### Re-format column names

In [ ]:
story_segment_df = eda_long_df[["segment_name", "subject_id"]].merge(
    extra_df[["ParticipantID", "STORY1NAME", "STORY2NAME", "STORY3NAME", "STORY4NAME", "STORY5NAME", "Story2_Score", "Story3_Score", "Story4_Score", "Story5_Score"]],
    left_on = ["subject_id"],
    right_on = ["ParticipantID"],
    how = "right"
)
# story_segment_df
story_segment_df.loc[:, "story_order"] = story_segment_df['segment_name']
story_segment_df.loc[:, "story_score"] = story_segment_df.apply(map_segment_story_score, axis=1)
story_segment_df.loc[:, "segment_name"] = story_segment_df.apply(map_segment_story_code, axis=1)
story_segment_df.head(3)

In [ ]:
cleaned_eda_long_df = (
    eda_long_df
    # 
    .merge(story_segment_df[["segment_name", "subject_id", "story_order", "story_score"]], 
           left_on=["segment_name", "subject_id"], 
           right_on=["story_order", "subject_id"],
           how="inner")
    .drop(columns=["segment_name_x"])
    .rename(columns = {"segment_name_y": "story_name"})
)
cleaned_eda_long_df.head(10)

### Pivot

In [ ]:
pivot_df = (
    cleaned_eda_long_df
    .pivot(index = "subject_id", columns=["story_name"], values=[
        "EDA_Tonic_Mean_baseline", "EDA_Tonic_Mean_blc", 
        "SCR_Peaks_N_per_seconds_baseline", "SCR_Peaks_N_per_seconds_blc", 
        "SCR_Peaks_Amplitude_Mean_baseline", "SCR_Peaks_Amplitude_Mean_blc", "story_score"])
    )
pivot_df

### Combine AC and BH into ACBH

In [ ]:
story_pair = ('AC', 'BH')
new_story = 'ACBH'

# Select only top-level columns (the metrics)
metrics = pivot_df.columns.levels[0]

for metric in metrics:
    if story_pair[0] in pivot_df[metric] or story_pair[1] in pivot_df[metric]:
        pivot_df[(metric, new_story)] = pivot_df[(metric, story_pair[1])].combine_first(
            pivot_df[(metric, story_pair[0])]
        )
        pivot_df = pivot_df.drop(columns=[(metric, story_pair[0]), (metric, story_pair[1])])
pivot_df.head(5)

### Prettify colnames

In [ ]:
eda_wide_df = prettify_pivot_colnames(pivot_df)
eda_wide_df

# Merge pivotted data with behavioral extra data

In [ ]:
final_df = (
    extra_df
    .merge(ecg_wide_df, left_on = "ParticipantID", right_on = "subject_id", how = "left")
    .merge(eda_wide_df.drop(columns = ['story_score_AS','story_score_NITP', 'story_score_SFT', 'story_score_SG', "story_score_ACBH"]), left_on = "ParticipantID", right_on = "subject_id", how = "left")
)

final_df

# Sanity checks

In [ ]:
ecg_long_df[["HRV_SDNN_baseline", "HRV_SDNN_corrected", 
        "RSA_PorgesBohrer_baseline", "RSA_PorgesBohrer_corrected",
        "heart_rate_bpm", "heart_rate_bpm_corrected", "segment_name"]].hist(bins=30, figsize=(20, 15))

In [ ]:
eda_long_df[["EDA_Tonic_Mean_baseline", "EDA_Tonic_Mean_blc", 
        "SCR_Peaks_N_per_seconds_baseline", "SCR_Peaks_N_per_seconds_blc", 
        "SCR_Peaks_Amplitude_Mean_baseline", "SCR_Peaks_Amplitude_Mean_blc", "segment_name"]].hist(bins=30, figsize=(20, 15))

# Export Final dataset

In [ ]:
final_df.to_excel(FINAL_DATA_DIR / "neuro-behavioral_data.xlsx", index = False)